# This file runs on ***Google Colab***
## Before running this file, the Data Source file needs to be placed at the same level as this file

### Data file name: PySpark_DataFile/hotel_reviews_for_PySpark_preprocessing.zip/hotel_reviews_for_PySpark_preprocessing.csv

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Cleaning_Data_for_PySpark").getOrCreate()

In [3]:
# Read in data 
from pyspark import SparkFiles
file_path = "hotel_reviews_for_PySpark_preprocessing.csv"
spark.sparkContext.addFile(file_path)
df = spark.read.csv(SparkFiles.get("hotel_reviews_for_PySpark_preprocessing.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+--------------+
|              Review|Reviewer_Score|
+--------------------+--------------+
| I am so angry th...|           2.9|
| No real complain...|           7.5|
| Rooms are nice b...|           7.1|
| My room was dirt...|           3.8|
| You When I booke...|           6.7|
| Backyard of the ...|           6.7|
| Cleaner did not ...|           4.6|
| Apart from the p...|          10.0|
| Even though the ...|           6.5|
| The aircondition...|           7.9|
| Nothing all grea...|          10.0|
| 6 30 AM started ...|           5.8|
| The floor in my ...|           4.6|
| This hotel is be...|           9.2|
| The staff in the...|           8.8|
| This hotel is aw...|          10.0|
| Very steep steps...|           6.3|
| We did not like ...|           7.5|
| Public areas are...|           7.1|
| We had issues wi...|           7.5|
+--------------------+--------------+
only showing top 20 rows



# Checking for Null, nan and empty strings

In [4]:
from pyspark.sql.functions import isnan, when, count, col

In [5]:
# Checking the df for empty strings
df.filter(df["Review"] == "").count()

0

In [6]:
# Checking the df for null values
df.filter(df["Review"].isNull()).count()

127

In [7]:
# Checking the df for nan values
df.filter(isnan(df["Review"])).count()

0

In [8]:
# Retaining only NotNull values
df = df.where(col("Review").isNotNull())

In [9]:
# Checking again for any null values
df.filter(df["Review"].isNull()).count()

0

In [10]:
# Exporting cleaned data as CSV
df.toPandas().to_csv('data_ready_for_PySpark_NLP.csv', index=False)